# Demo: Diskrete Kosinus-Transformation

Abkürzung: DCT = Discrete Cosine Transformation

## Bibliotheken

In [ ]:
import numpy as np
import matplotlib.pyplot as pl
import cv2 as cv

path = '/exchange/cvai/images/'

## Eindimensionale DCT

### Kosinusfunktionen im Intervall $[0,\pi]$

In [ ]:
Farbe = ['red', 'green', 'turquoise', 'blue', 'violet']

In [ ]:
def KosinusKurven (N):
    x = np.linspace (0, np.pi, 1001)
    for n in range (N):
        pl.plot (x, np.cos (n * x), color = Farbe [n])

KosinusKurven (3)

### Diskrete Kosinusfunktionen

In [ ]:
def KosinusKurvenPunkte (N):
    x1 = np.linspace (0, np.pi, 1001)
    x2 = (2 * np.arange (N) + 1) / (2 * N) * np.pi
    for n in range (N):
        pl.plot (x1, np.cos (n * x1), color = Farbe [n])
        pl.plot (x2, np.cos (n * x2), 'o', color = Farbe [n])

KosinusKurvenPunkte (3)

### Kosinustransformation als Matrix

#### Stützstellen

In [ ]:
def DctArgs (N):
    jj = 2 * np.arange (N) + 1
    tt = jj / (2 * N)
    xx = np.pi * tt
    return xx

DctArgs (2)

#### Eine Zeile

In [ ]:
def DctZeile (N, n):
    xx = DctArgs (N)
    return np.cos (n * xx)

print (DctZeile (2, 0))
print (DctZeile (2, 1))

#### Vollständige Matrix

In [ ]:
def DctMatrix (N):
    C = np.zeros ((N, N))
    for n in range (N):
        C [n, :] = DctZeile (N, n)
    C /= np.sqrt (N); C [1:, :] *= np.sqrt (2)
    return C

M = DctMatrix (2)
print (M)

#### Orthogonalität

In [ ]:
print (M.T @ M)

### Transformation angewandt auf eine diskretisierte Funktion

In [ ]:
def Test (f, N):
    x = DctArgs (N)
    y = f (x)
    C = DctMatrix (N)
    print (C @ y)
    
    xx = np.linspace (0, np.pi, 1001)
    pl.figure (figsize = (10, 5))
    pl.plot (xx, f (xx), color = 'gray', linewidth = 3)
    pl.plot (x, y, 'o', color = 'gray')
    KosinusKurvenPunkte (N)

Test (lambda x: np.cos (1.1 * x) + 0.1, 4)

## Zweidimensionale DCT

### Eine Basisfunktion aus einem Bild mit einem hellen Pixel

In [ ]:
def EinPixelBild (N, k, l):
    X = np.zeros ((N, N))
    X [k, l] = 1
    return X

def EineBasis (N, k, l):
    X = EinPixelBild (N, k, l)
    C = DctMatrix (N)
    pl.figure (figsize = (10, 5))
    pl.subplot (1, 2, 1); pl.imshow (X, cmap = 'gray')
    pl.subplot (1, 2, 2); pl.imshow (C.T @ X @ C, cmap = 'gray')
    return X

t = EineBasis (8, 3, 2)

### Alle Basisfunktionen

In [ ]:
def AlleBasen (N):
    i = 0
    C = DctMatrix (N)
    for j in range (N):
        for k in range (N):
            i = i + 1
            pl.subplot (N, N, i)
            X = EinPixelBild (N, j, k)
            pl.imshow (C.T @ X @ C, cmap = 'gray')

pl.figure (figsize = (10, 10))
AlleBasen (8)

## Anwendung: Bildkompression

### Bildquelle

https://breckon.org/fundipbook/materials/

### Bild einlesen und darstellen

In [ ]:
X = cv.imread (path + 'autumn.png')
X = cv.cvtColor (X, cv.COLOR_BGR2GRAY)

In [ ]:
pl.imshow (X, cmap = 'gray')

### Information zu einem Bild ausgeben

In [ ]:
def info (im):
    print ('shape:', im.shape, '/ min:', np.min (im), '/ max:', np.max (im))

In [ ]:
info (X)

### Intensitäten "verschieben"

In [ ]:
Min = np.min (X)
Max = np.max (X)
X1 = 1.0 * (X - Min) / (Max - Min) - 0.0
info (X1)

### Bild in den Frequenzraum transformieren

#### Transformationsmatrizen

In [ ]:
M, N = X1.shape
CM = DctMatrix (M)
CN = DctMatrix (N)
info (CM)
info (CN)

#### Transformation durchführen

In [ ]:
Y1 = CM @ X1 @ CN.T
info (Y1)

#### Test: Rücktransformation

In [ ]:
tmp = CM.T @ Y1 @ CN
info (tmp)

#### Hohe Frequenzen "abschneiden"

In [ ]:
Y2 = np.copy (Y1)
Y2 [30:, 30:] = 0
info (Y2)

#### Bild mit abgeschnittenen Frequenzen rücktransformieren

In [ ]:
Z2 = CM.T @ Y2 @ CN
info (X1)
info (Z2)

#### Ursprüngliches und komprimiertes Bild darstellen

In [ ]:
pl.figure (figsize = (14, 7))
pl.subplot (1, 3, 1); pl.imshow (X1, cmap = 'gray')
info (X1)
pl.subplot (1, 3, 2); pl.imshow (Z2, cmap = 'gray')
info (Z2)
pl.subplot (1, 3, 3); pl.imshow (X1 - Z2, cmap = 'gray')
info (X1 - Z2)